In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import gc
!pip install segmentation-models
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
trainImgPath = "/kaggle/input/severstal-steel-defect-detection/train_images/"
testImgPath = "/kaggle/input/severstal-steel-defect-detection/test_images/"
trainCsv = "/kaggle/input/severstal-steel-defect-detection/train.csv"
data=pd.read_csv(trainCsv)
data.ClassId=data.ClassId.astype(int)

In [ ]:
WIDTH=800
HEIGHT=128 

In [ ]:
def convert_to_mask(encoded_pixels):
    counts=[]
    mask=np.zeros((256*1600), dtype=np.uint8) #don't change this
    pre_mask=np.asarray([int(point) for point in encoded_pixels.split()])
    for index,count in enumerate(pre_mask):
        if(index%2!=0):
            counts.append(count)
    i=0
    for index,pixel in enumerate(pre_mask):
        if(index%2==0):
            if(i==len(counts)):
                break
            mask[pixel:pixel+counts[i]]=1
            i+=1
    mask=np.reshape(mask,(1600,256)) #don't change this
    mask=cv2.resize(mask,(HEIGHT,WIDTH)).T
    return mask

In [ ]:
import keras 
class Data_Generator(keras.utils.Sequence):
    
    def __init__(self, x_set, y_set, batch_size=16,preprocess=None,shuffle=False,op="train",info={},width=WIDTH,height=HEIGHT):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.preprocess = preprocess
        self.shuffle = shuffle
        self.info=info
        self.h = height
        self.w = WIDTH
        self.op = op
        if self.op == "train":
            self.path = trainImgPath
        elif self.op == "test":
            self.path = testImgPath 
        self.on_epoch_end()    
        
        

    def __len__(self):
        return int(np.floor(len(self.x) / self.batch_size))
    
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.x))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
        
        
    def __getitem__(self, idx):
        imgs = np.empty((self.batch_size,self.h,self.w,3),dtype=np.float32)
        masks = np.empty((self.batch_size,self.h,self.w,4),dtype=np.int8)
        indexes = self.indexes[idx*self.batch_size: (idx+1)*self.batch_size]
        
        for index , val in enumerate(self.x.iloc[indexes]):
            self.info[index*self.batch_size+index] = val
            img=cv2.imread(self.path + val) 
            imgs[index,] = cv2.resize(img,(self.w,self.h))
            if self.op == "train":
                for idx in range(4):
                    masks[index,:,:,idx] = convert_to_mask(self.y['Defect_'+str(idx+1)].iloc[indexes[index]])
        if self.preprocess :
            imgs = self.preprocess(imgs)
        if self.op == "train":
            return imgs,masks 
        else: 
            return imgs
            
            

In [ ]:
train_Img_Id = []
train_class_Id = []
for i in os.listdir(trainImgPath):
    for j in range(1,5):
        train_Img_Id.append(i)
        train_class_Id.append(j)
train_Imgs = pd.DataFrame(train_Img_Id,columns=['ImageId'])
train_Imgs['ClassId'] = train_class_Id
train_Imgs.head(10)

In [ ]:
train_data = pd.merge(train_Imgs,data ,how='outer', on=['ImageId','ClassId']) 
train_data = train_data.fillna('') 
train_data.head(10)

In [ ]:
train_data = pd.pivot_table(train_data, values='EncodedPixels', index='ImageId',columns='ClassId', aggfunc=np.sum).astype(str)
train_data = train_data.reset_index() # add Index column to one level with classID   
train_data.columns = ['ImageId','Defect_1','Defect_2','Defect_3','Defect_4']
train_data.columns

In [ ]:
has_defect = []
stratify = []
for index,row in train_data.iterrows():
    if row.Defect_1 or row.Defect_2 or row.Defect_3 or row.Defect_4: 
        has_defect.append(1)
    else:
        has_defect.append(0)
        
train_data["has_defect"] = has_defect 
 
 
for index , row in train_data.iterrows():
    if row.Defect_1 != '':
        stratify.append(1)
    elif row.Defect_2 != '':
        stratify.append(2)
    elif row.Defect_3 != '':
        stratify.append(3)
    elif row.Defect_4 != '':
        stratify.append(4)
    else:
        stratify.append(0)
        
train_data["stratify"] = stratify   
train_data.head(5)

In [ ]:
xset = train_data["ImageId"]
yset = train_data.iloc[:,1:5]
xset[0] 

In [ ]:
x_train, x_test = train_test_split(train_data, test_size = 0.1, stratify=train_data['stratify'], random_state=42)
x_train, x_val = train_test_split(x_train, test_size = 0.2, stratify = x_train['stratify'], random_state=42)
print(x_train.shape, x_val.shape, x_test.shape)

In [ ]:
x_train_all = x_train[['ImageId','has_defect']]
x_val_all = x_val[['ImageId','has_defect']]
x_test_all = x_test[['ImageId','has_defect']] 
print(x_train_all.shape , x_val_all.shape,x_test_all.shape)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator 

train_datagen = ImageDataGenerator(rescale=1./255., shear_range=0.2, zoom_range=0.05, rotation_range=5,
                           width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data_generator = train_datagen.flow_from_dataframe(
        dataframe=x_train_all.astype(str),
        directory=trainImgPath,
        x_col="ImageId",
        y_col="has_defect",
        target_size=(HEIGHT,WIDTH),
        batch_size=16,
        class_mode='binary') 

valid_data_generator = test_datagen.flow_from_dataframe(
        dataframe=x_val_all.astype(str),
        directory=trainImgPath,
        x_col="ImageId",
        y_col="has_defect",
        target_size=(HEIGHT,WIDTH),
        batch_size=16,
        class_mode='binary')



In [ ]:
epochs = 50 
from tensorflow.keras.utils import plot_model
import keras 
from keras import backend as K
from keras.layers import GlobalAveragePooling2D, Dense, Conv2D, BatchNormalization, Dropout
from keras.models import Model, load_model



Classification_Model = keras.applications.xception.Xception(include_top = False, input_shape = (HEIGHT,WIDTH,3))

layer = Classification_Model.output
layer = GlobalAveragePooling2D()(layer)

layer = Dense(1024, activation='relu')(layer)
layer = BatchNormalization()(layer)
layer = Dropout(0.3)(layer)

layer = Dense(512, activation='relu')(layer)
layer = BatchNormalization()(layer)
layer = Dropout(0.3)(layer)

layer = Dense(64, activation='relu')(layer)
predictions = Dense(1, activation='sigmoid')(layer)
model = Model(inputs=Classification_Model.input, outputs=predictions)
model.summary()

In [ ]:
import h5py

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model_history = model.fit_generator(train_data_generator, validation_data = valid_data_generator, epochs = 30, verbose=1)
model.save('classification_model.h5')

In [ ]:
model.save('classification') 


In [ ]:
test_data_generator = test_datagen.flow_from_dataframe(
        dataframe=x_test_all.astype(str),
        directory=trainImgPath,
        x_col="ImageId",
        y_col="has_defect",
        target_size=(HEIGHT,WIDTH),
        batch_size=16,
        class_mode='binary',
        shuffle=False)


In [ ]:
test_evaluate = model.evaluate(test_data_generator,verbose=1)


In [ ]:
test_evaluate

In [ ]:
from keras import backend as K
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import jaccard_loss
from segmentation_models.metrics import iou_score
 
network = 'resnet34'
process_input = get_preprocessing(network)
 
model = Unet(network,input_shape = (HEIGHT, WIDTH, 3),classes=4,activation='sigmoid')
model.compile('adam', loss='binary_crossentropy',metrics=[dice_coef])


In [ ]:
idx = int(0.8*len(xset)) #dropout 0.2 of dataset 
train_data_generator = Data_Generator(xset.iloc[:idx],yset.iloc[:idx],shuffle=True,preprocess=process_input )
 
val_data_generator = Data_Generator(xset.iloc[:idx],yset.iloc[:idx],preprocess=process_input)


In [ ]:
model.fit_generator(
   train_data_generator, 
   validation_data = val_data_generator, 
   epochs = 30,
   verbose=2
)

In [ ]:
model.save('segmentation')

In [ ]:
results=model.evaluate(xtest,ytest)
resullts

In [ ]:
model.save('segmentation')